<a href="https://colab.research.google.com/github/ilucas75/BTflow/blob/dev/RD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#install
!pip install pdfminer.six
!pip install spacy

In [5]:
!python -m spacy download fr_core_news_sm
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 14.7MB 1.8MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=b6eabca1141a4df748bd35a2972b6a8eda2e286307cbe151e69041299a7e1dc8
  Stored in directory: /tmp/pip-ephem-wheel-cache-wxo507d3/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
     |████████████████████████████████| 88.6MB 1.1MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp36-none-any.whl size=90338488 sha256=a17e89b1eeaf08cbf032158457756fd9a328127f3e273988b340f1f84d410fb4
  Stored in directory: /tmp/pip-ephem-wheel-cache-sm7bdxpg/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [6]:
from google.colab import files

uploaded = files.upload()


Saving 0010C022_220C0100.pdf to 0010C022_220C0100.pdf
Saving 0020C022_220C0200.pdf to 0020C022_220C0200.pdf
Saving 0030C022_220C0300.pdf to 0030C022_220C0300.pdf
Saving 0040C022_220C0400.pdf to 0040C022_220C0400.pdf
Saving 0050C022_220C0500.pdf to 0050C022_220C0500.pdf
Saving 0060C022_220C0600.pdf to 0060C022_220C0600.pdf
Saving 0070C022_220C0700.pdf to 0070C022_220C0700.pdf
Saving 0080C022_220C0800.pdf to 0080C022_220C0800.pdf
Saving 0090C022_220C0900.pdf to 0090C022_220C0900.pdf
Saving 0100C022_220C0010.pdf to 0100C022_220C0010.pdf
Saving 0110C022_220C0110.pdf to 0110C022_220C0110.pdf
Saving 0120C022_220C0210.pdf to 0120C022_220C0210.pdf
Saving 0130C022_220C0310.pdf to 0130C022_220C0310.pdf
Saving 0140C022_220C0410.pdf to 0140C022_220C0410.pdf
Saving 0150C022_220C0510.pdf to 0150C022_220C0510.pdf
Saving 0160C022_220C0610.pdf to 0160C022_220C0610.pdf
Saving 0170C022_220C0710.pdf to 0170C022_220C0710.pdf
Saving 0180C022_220C0810.pdf to 0180C022_220C0810.pdf
Saving 0190C022_220C0910.pdf

In [1]:
ls

sample_data/


In [0]:
import io
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [0]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

In [40]:
pdfs = []
for f in uploaded.keys():
  try:
    text=extract_text_from_pdf(f)
    if "Déclaration de franchissement de seuils" in text:
      pdfs.append({
          "file": f,
          "text": text,
      })
  except:
    pass

ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [0]:
import json
with open('dataset.json') as f:
  pdfs = json.load(f)

In [42]:
import IPython
from google.colab import output

class AnnotWidget():
  def __init__(self, to_annot):
    self.to_annot = to_annot
    self.cur = 0

  def run(self):
    display(IPython.display.HTML('''
    <div id="ed" contenteditable="true">'''+self.to_annot[self.cur]["text"]+'''</div>
    <fieldset>
      <button onclick="document.execCommand('backColor', false, '#ff0000')">issuer</button>
      <button onclick="document.execCommand('backColor', false, '#ff00ff')">firm</button>
      <button onclick="document.execCommand('backColor', false, '#00ffff')">number</button>
      <button onclick="document.execCommand('backColor', false, '#a0a0ff')">percent</button>
      <button onclick="document.execCommand('backColor', false, '#c0c0c0')">type</button>
      <button onclick="document.execCommand('backColor', false, '#ffff00')">direction</button>
      <button onclick="document.execCommand('backColor', false, '#00ff00')">date</button>
      <button id='save'>Save</button>
    </fieldset>
    <script>
      document.querySelector('#save').onclick = () => {
        colors_label = {"rgb(255, 0, 0)" : "ISSUER", "rgb(255, 0, 255)": "FIRM", "rgb(0, 255, 255)" : "NUMBER",
         "rgb(160, 160, 255)" : "PERCENT", "rgb(192, 192, 192)" : "TYPE", "rgb(255, 255, 0)" : "DIRECTION",
         "rgb(0, 255, 0)" : "DATE"};
        children = document.querySelector("#ed").childNodes;
        res = [];
        offset = 0;
        for (var i = 0; i < children.length; i++) {
          next_offset = offset + children[i].textContent.length;
          if (children[i].nodeType != Node.TEXT_NODE) {
            rgb = children[i].style.backgroundColor
            res.push([offset, next_offset, colors_label[rgb]])
            //google.colab.kernel.invokeFunction('notebook.AddListItem', [offset, next_offset, colors_label[rgb]], {});
          }
          offset = next_offset;
        }
        next_text = google.colab.kernel.invokeFunction('notebook.SaveEntities', [res]);
        document.querySelector('#ed').innerHTML = next_text;
      };
    </script>
    '''))
    output.register_callback('notebook.SaveEntities', self.save_entities)

  def save_entities(self, res):
    self.to_annot[self.cur]['entities'] = res
    output.clear()
    self.cur += 1
    if self.cur < len(self.to_annot):
      self.run()
    else:
      return self.to_annot


w = AnnotWidget(pdfs) 
w.run()

In [0]:
import json
with open("dataset.json", 'w+') as f:
  json.dump(w.to_annot, f)

In [0]:
entities = [tuple(ent) for ent in w.to_annot[0]["entities"]]
train_data = [(w.to_annot[0]['text'], {'entities' : entities})]
labels = ['ISSUER', 'FIRM', 'NUMBER', 'PERCENT', 'TYPE', 'DIRECTION', 'DATE']

In [51]:
import random
import spacy
from spacy.util import minibatch, compounding
import fr_core_news_md

n_iter=30
nlp = fr_core_news_md.load()
ner = nlp.get_pipe("ner")
for label in labels:
  ner.add_label(label)
optimizer = nlp.resume_training()
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train NER
    sizes = compounding(1.0, 4.0, 1.001)
    # batch up the examples using spaCy's minibatch
    for itn in range(n_iter):
        random.shuffle(train_data)
        batches = minibatch(train_data, size=sizes)
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
        print("Losses", losses)

Losses {'ner': 499.8311448097229}
Losses {'ner': 413.8562733530998}
Losses {'ner': 413.6670391559601}
Losses {'ner': 459.4286390542984}
Losses {'ner': 397.6553840637207}
Losses {'ner': 386.09096574783325}
Losses {'ner': 411.8200770020485}
Losses {'ner': 425.5337452888489}
Losses {'ner': 402.93844985961914}
Losses {'ner': 389.51711654663086}
Losses {'ner': 400.52786219120026}
Losses {'ner': 379.5752320289612}
Losses {'ner': 379.3474712371826}
Losses {'ner': 370.37407970428467}
Losses {'ner': 356.06399643421173}
Losses {'ner': 364.74296593666077}
Losses {'ner': 328.357519865036}
Losses {'ner': 374.6704339981079}
Losses {'ner': 343.4950132369995}
Losses {'ner': 344.44360268116}
Losses {'ner': 340.8940062522888}
Losses {'ner': 370.98770809173584}
Losses {'ner': 367.88058400154114}
Losses {'ner': 355.09017968177795}
Losses {'ner': 333.72866678237915}
Losses {'ner': 323.12162017822266}
Losses {'ner': 324.1234347820282}
Losses {'ner': 328.60198616981506}
Losses {'ner': 327.05132722854614}
Los

In [52]:
doc = nlp(train_data[0][0])
for ent in doc.ents:
    print(ent.label_, ent.text)

PER Wilmington
PER Merrill Lynch International
ORG Inc
ORG Merrill


In [16]:
!pip install tabula-py

     |████████████████████████████████| 10.4MB 3.3MB/s 


In [20]:
from tabula import read_pdf

list_pdf = list(pdfs.keys())
df = read_pdf(list_pdf[0], pages='all')

Got stderr: Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [29]:
import pandas as pd
for d in df:
  if not d.empty:
    print(d)
    table_str = ""
    for c in d.columns:
      if not "Unnamed:" in c:
        table_str += c + " "
    for _, r in d.iterrows():
      for c in d.columns:
        if pd.notna(r[c]):
          table_str += r[c] + " "
    print(table_str)

                          Unnamed: 0  Unnamed: 1  ... % droits de vot Unnamed: 10
0        Merrill Lynch International  14 101 661  ...             NaN         NaN
1          BofA Securities Europe SA     276 021  ...             NaN         NaN
2              BofA Securities, Inc.         213  ...             NaN         NaN
3  Total Bank of America Corporation  14 377 895  ...             NaN         NaN

[4 rows x 15 columns]
Actions % capita Droits de vote % droits de vot Merrill Lynch International 14 101 661 5,49 14 101 661 5,47 BofA Securities Europe SA 276 021 0,11 276 021 0,11 BofA Securities, Inc. 213 ns 213 ns Total Bank of America Corporation 14 377 895 5,59 14 377 895 5,58 


In [0]:
import json
with open("texts.json", "w+") as f:
  json.dump(pdfs, f)


In [0]:
with open("texts.txt", "w+") as f:
  f.write(pdfs[list_pdf[0]]['text'])

In [59]:
import spacy
import fr_core_news_sm
import fr_core_news_md

#nlp = fr_core_news_sm.load()
#nlp = fr_core_news_md.load()

doc = nlp(train_data[0][0])
for sent in doc.sents:
    print(sent)



220C0700-FR0000034639-FS0185            220C0700
FR0000034639-FS0185 21 février 2020   
Déclaration de franchissement de seuils (article L. 233
-7 du code de commerce)   ALTRAN TECHNOLOGIES  (Euronext Paris)   Par courrier reçu le 20 février 2020, complété par un courrier reçu le 21 février, la société Bank of America Corporation (Corporation Trust Center, Orange Street, Wilmington, DE 19801, Etats-Unis) a déclaré avoir franchi en hausse, indirectement par l’intermédiaire des sociétés qu’elle contrôle, le 17 février 2020, les seuils de 5% du capital et des droits de vote de la société ALTRAN TECHNOLOGIES et détenir 14 377 895 actions ALTRAN TECHNOLOGIES représentant autant de droits de vote, soit 5,59% du capital et 5,58% des droits de vote de cette société1, répartis comme suit :   Actions % capital Droits de vote % droits de vote Merrill Lynch International 14 101 661 5,49 14 101 661 5,47 BofA Securities Europe SA 276 021 0,11 276 021 0,11 BofA Securities, Inc.
213 ns 213 ns Total Ba

In [63]:
for i in range(len(train_data[0][1]['entities'])):
  s, e, _ = train_data[0][1]['entities'][i]
  print(train_data[0][0][s:e])


21 février 2020
ALTRAN TECHNOLOGIES
20 février 2020
21 février
Bank of America Corporation
en hausse
17 février 2020
5%
capital
droits de vote
ALTRAN TECHNOLOGIES
14 377 895
actions
ALTRAN TECHNOLOGIES
5,59%
capital
5,58%
droits de vote
ALTRAN TECHNOLOGIES
ALTRAN TECHNOLOGIES
Merrill Lynch International
en hausse
Merrill Lynch International
13 784 025
actions
ALTRAN TECHNOLOGIES
Merrill Lynch International
BofA Securities Europe SA
274 482
actions
ALTRAN TECHNOLOGIES
BofA Securities Europe SA
Merrill Lynch International
17 749
actions
ALTRAN TECHNOLOGIES


In [0]:
from spacy import displacy

In [61]:
displacy.render(doc, style='ent', jupyter=True)

In [0]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.label_, chunk.root.text)
 

220C0170-FR0012407096-OP028-AS12-RO02            220C0170 FR0012407096-OP028-AS12-RO02 14 janvier 2020    NP 220C0170-FR0012407096-OP028-AS12-RO02
- Mise en œuvre du retrait obligatoire visant les actions de la société.   NP Mise
- Maintien de la suspension de la cotation des actions de la société.     NP Maintien
Euronext Paris NP Euronext
Le 13 janvier 2020 NP janvier
l’Autorité des marchés financiers NP Autorité
d’achat simplifiée NP achat
les actions ITS GROUP NP actions
la société ITS Participations NP société
actions ITS GROUP NP actions
de droits de vote NP droits
des droits de vote de cette société2 NP droits
cf NP cf
D&I 220C0149 en date du 13 janvier 2020).   NP D&I
Les actions ITS GROUP non présentées à l’offre par les actionnaires minoritaires NP actions
de 366 396 actions NP actions
483 droits de vote, NP droits
soit 4,65% du capital et 4,92% des droits de vote de cette société2 NP %
2- Par courrier NP 2-
de la société NP société
ITS Participations NP Participations
l’Auto